In [1]:
import zipfile
import geopandas as gpd
import logging
import os
import locale
import requests
from ee_s1_ard import S1ARDImageCollection
import pandas as pd
import ee 
ee.Authenticate()
ee.Initialize(project='ee-caiosimplicioarantes') 

#from ravi_ee_tools import load_aoi_from_shapefile # Assuming your_module_name.py contains the function
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
app_logger = logging.getLogger(__name__)
logger = logging.getLogger(__name__)

In [2]:
def load_aoi_from_shapefile(shapefile_path):
    """
    Loads an Area of Interest (AOI) from a shapefile (or a zip containing a shapefile)
    and converts it into an Earth Engine FeatureCollection.

    Args:
        shapefile_path (str): The path to the shapefile (.shp) or a zip archive
                              containing a .shp file.

    Returns:
        ee.FeatureCollection: An Earth Engine FeatureCollection representing the AOI.

    Raises:
        FileNotFoundError: If no .shp file is found inside a zip archive.
        ValueError: If the shapefile does not contain any geometries.
        Exception: For other errors during shapefile reading or processing.
    """
    gpd_aoi = None # Use a distinct name for the geopandas DataFrame

    if shapefile_path.endswith(".zip"):
        with zipfile.ZipFile(shapefile_path, "r") as zip_ref:
            shapefile_within_zip = None
            for file in zip_ref.namelist():
                if file.lower().endswith(".shp"): # Use .lower() for case-insensitivity
                    shapefile_within_zip = file
                    break
            if not shapefile_within_zip:
                logger.error(f"No .shp file found inside the zip archive: {shapefile_path}")
                raise FileNotFoundError(
                    f"No .shp file found inside the zip archive: {shapefile_path}"
                )

            # Read shapefile directly from the zip archive.
            gpd_aoi = gpd.read_file(f"zip://{shapefile_path}/{shapefile_within_zip}")
    else:
        gpd_aoi = gpd.read_file(shapefile_path)

    # Reproject the GeoDataFrame to EPSG:4326 to ensure correct
    # coordinates for Earth Engine.
    gpd_aoi = gpd_aoi.to_crs(epsg=4326)

    if gpd_aoi.empty:
        logger.error(f"The shapefile at {shapefile_path} does not contain any geometries.")
        raise ValueError(f"The shapefile at {shapefile_path} does not contain any geometries.")

    # Dissolve multiple features into a single geometry if necessary.
    if len(gpd_aoi) > 1:
        logger.info("Multiple features found; dissolving into a single geometry.")
        gpd_aoi = gpd_aoi.dissolve()

    # Extract the first geometry.
    # It's safer to ensure there's at least one geometry before iloc[0]
    if gpd_aoi.empty: # Re-check after dissolve if it became empty (unlikely but robust)
         logger.error(f"The shapefile at {shapefile_path} became empty after dissolve.")
         raise ValueError(f"The shapefile at {shapefile_path} became empty after dissolve.")

    geometry = gpd_aoi.geometry.iloc[0]

    # Convert the geometry to GeoJSON format.
    geojson = geometry.__geo_interface__

    # Remove any third dimension from the coordinates.
    # Note: GeoJSON can support 3D, but Earth Engine often expects 2D.
    # This part of the logic is robust.
    if geojson["type"] == "Polygon":
        geojson["coordinates"] = [
            list(map(lambda coord: coord[:2], ring)) for ring in geojson["coordinates"]
        ]
    elif geojson["type"] == "MultiPolygon":
        geojson["coordinates"] = [
            [list(map(lambda coord: coord[:2], ring)) for ring in polygon]
            for polygon in geojson["coordinates"]
        ]

    # Create an Earth Engine geometry object.
    ee_geometry = ee.Geometry(geojson)
    feature = ee.Feature(ee_geometry)
    ee_feature_collection = ee.FeatureCollection([feature])

    logger.info("AOI defined successfully.")
    return ee_feature_collection

In [3]:
aoi = load_aoi_from_shapefile('contorno_area_total.zip')

2025-08-10 19:30:08,972 - __main__ - INFO - AOI defined successfully.


In [4]:
start_date = '2025-02-09'
stop_date = '2025-08-09'

In [5]:
processor = S1ARDImageCollection(
    geometry=aoi,
    start_date=start_date,
    stop_date=stop_date,
    polarization="VVVH",
    apply_border_noise_correction=True,
    apply_terrain_flattening=True,
    apply_speckle_filtering=True,
    output_format="DB",
    ascending=False # False for descending orbit, True for ascending orbit
)

collection = processor.get_collection().sort("system:time_start", False)  # Sort by time in descending order

collection.size().getInfo()  # This will trigger the processing and return the size of the collection

2025-08-10 19:30:13,704 - googleapiclient.http - WARNING - Sleeping 0.72 seconds before retry 1 of 5 for request: POST https://earthengine.googleapis.com/v1/projects/ee-caiosimplicioarantes/value:compute?prettyPrint=false&alt=json, after ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))


24

In [6]:
def add_vvvh_ratio_band(image):
    ratio = image.select("VV").divide(image.select("VH")).rename("VVVH_ratio")
    return image.addBands(ratio)

collection = collection.map(add_vvvh_ratio_band)

In [7]:
# Extract the VVVH_ratio band and reduce over the geometry to get mean values for each image
def get_vvvh_ratio_mean(image):
    stats = image.select("VVVH_ratio").reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=10,
        maxPixels=1e9
    )
    # Get image date
    date = image.date().format('YYYY-MM-dd')
    return ee.Feature(None, {
        'date': date,
        'VVVH_ratio_mean': stats.get('VVVH_ratio')
    })

# Map over the collection to get time series
vvvh_ratio_ts = collection.map(get_vvvh_ratio_mean).getInfo()

# Convert to pandas DataFrame for easier handling

data = [
    {'dates': f['properties']['date'], 'AOI_average': f['properties']['VVVH_ratio_mean']}
    for f in vvvh_ratio_ts['features']
]

df = pd.DataFrame(data)

In [8]:
import plotly.express as px

fig = px.line(df, x='dates', y='AOI_average', markers=True, title='VV/VH Ratio Mean Time Series')
fig.update_layout(xaxis_title='Date', yaxis_title='VV/VH Ratio Mean')
fig.show()

In [10]:
from datetime import datetime, timedelta
date = df.dates[3]
next_date = (datetime.strptime(date, "%Y-%m-%d") + timedelta(days=1)).strftime("%Y-%m-%d")
selected_image = collection.filterDate(date, next_date).first().select(['VV', 'VH', 'VVVH_ratio']).clip(aoi)

In [11]:
try:
    # Prepare download URL for the clipped image
    url = selected_image.getDownloadURL({
        "scale": 10,
        "region": aoi.geometry().bounds().getInfo(),
        "format": "GeoTIFF",
        "crs": "EPSG:4326"
    })

    output_file = f"Sentinel1_{date}.tiff"

    response = requests.get(url)
    if response.status_code == 200:
        with open(output_file, "wb") as f:
            f.write(response.content)
        print(f"Image downloaded as {output_file}")
    else:
        print(f"Failed to download image. HTTP Status: {response.status_code}")
except Exception as e:
    print(f"An error occurred: {e}")

Image downloaded as Sentinel1_2025-07-18.tiff
